In [1]:
import pandas as pd
from db_connect import get_db_connection
from PyPDF2 import PdfReader

# Importer les aerports

In [3]:
airport_data = pd.read_excel('data/airports.xlsx')
airport_data = airport_data.drop(columns=['Unnamed: 0'])
airport_data

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
...,...,...,...,...,...,...,...,...
1453,ZUN,Black Rock,35.083228,-108.791778,6454,-7,A,America/Denver
1454,ZVE,New Haven Rail Station,41.298669,-72.925992,7,-5,A,America/New_York
1455,ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0,-5,A,America/New_York
1456,ZWU,Washington Union Station,38.897460,-77.006430,76,-5,A,America/New_York


# Insérez les données dans la table airport

In [4]:
data_to_insert = [
    (
        row['faa'], row['name'], row['lat'], row['lon'], row['alt'], row['tz'], row['dst'], row['tzone']
    )
    for index, row in airport_data.iterrows()
]
connection = get_db_connection()
cursor = connection.cursor()
cursor.executemany(
    "INSERT INTO airport (faa, name, lat, lon, alt, tz, dst, tzone) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
    data_to_insert
)
connection.commit()
connection.close()

Connexion réussie à la base de données SQL Server.


# Importer les vols

In [5]:
flights_data = pd.read_excel('data/flights.xlsx', header=None)
flights_data = flights_data[0].str.split(',', expand=True) # Permet de séparer les colonnes
flights_data.columns = flights_data.iloc[0] # Permet de mettre la première ligne en header
flights_data = flights_data[1:]
flights_data

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1,2021,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2021-01-01T05:00:00Z
2,2021,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2021-01-01T05:00:00Z
3,2021,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2021-01-01T05:00:00Z
4,2021,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2021-01-01T05:00:00Z
5,2021,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2021-01-01T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252700,2021,7,3,1059,1100,-1,1214,1210,4,MQ,3230,N522MQ,JFK,DCA,51,213,11,0,2021-07-03T11:00:00Z
252701,2021,7,3,1059,1100,-1,1207,1213,-6,US,2171,N754UW,LGA,DCA,43,214,11,0,2021-07-03T11:00:00Z
252702,2021,7,3,1101,1110,-9,1305,1240,25,9E,3910,N813AY,JFK,RIC,61,288,11,10,2021-07-03T11:00:00Z
252703,2021,7,3,1105,1055,10,1435,1358,37,B6,971,N598JB,LGA,FLL,176,1076,10,55,2021-07-03T10:00:00Z


# Insérez les données dans la table flight

In [6]:
flights_data['time_hour'] = pd.to_datetime(flights_data['time_hour'])
data_to_insert = [
    (
         row['year'], row['month'], row['day'], row['dep_time'], row['sched_dep_time'], row['dep_delay'], row['arr_time'], row['sched_arr_time'], row['arr_delay'], row['carrier'], row['flight'], row['tailnum'], row['origin'], row['dest'], row['air_time'], row['distance'], row['hour'], row['minute'], row['time_hour']
    )
    for index, row in flights_data.iterrows()
]

# Insertion base de données
connection = get_db_connection()
cursor = connection.cursor()
cursor.executemany(
    "INSERT INTO flight (year, month, day, dep_time, sched_dep_time, dep_delay, arr_time, sched_arr_time, arr_delay, carrier, flight, tailnum, origin, dest, air_time, distance, hour, minute, time_hour)"
    "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    data_to_insert
)
connection.commit()
connection.close()

Connexion réussie à la base de données SQL Server.


In [ ]:
# Importer les compagnies

In [7]:
df_airlines = pd.read_json('data/airlines.json')
df_airlines

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.
5,EV,ExpressJet Airlines Inc.
6,F9,Frontier Airlines Inc.
7,FL,AirTran Airways Corporation
8,HA,Hawaiian Airlines Inc.
9,MQ,Envoy Air


# Insérez les données dans la table airline

In [8]:
data_to_insert = [
    (
        row['carrier'], row['name']
    )
    for index, row in df_airlines.iterrows()
]

connection = get_db_connection()
cursor = connection.cursor()

cursor.executemany(
    "INSERT INTO airline (carrier, name) VALUES (?, ?)",
    data_to_insert
)

connection.commit()
connection.close()

Connexion réussie à la base de données SQL Server.


# Importer les avions

In [9]:
df_planes = pd.read_html('data/planes.html')[0]

df_planes['year'] = df_planes['year'].fillna(0).astype(int)

df_planes['speed'] = df_planes['speed'].fillna(0)

df_planes

,Unnamed: 0,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,0.0,Turbo-fan
1,1,N102UW,1998,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,0.0,Turbo-fan
2,2,N103US,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,0.0,Turbo-fan
3,3,N104UW,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,0.0,Turbo-fan
4,4,N10575,2002,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,0.0,Turbo-fan
...,...,...,...,...,...,...,...,...,...,...
3317,3317,N997AT,2002,Fixed wing multi engine,BOEING,717-200,2,100,0.0,Turbo-fan
3318,3318,N997DL,1992,Fixed wing multi engine,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,2,142,0.0,Turbo-fan
3319,3319,N998AT,2002,Fixed wing multi engine,BOEING,717-200,2,100,0.0,Turbo-fan
3320,3320,N998DL,1992,Fixed wing multi engine,MCDONNELL DOUGLAS CORPORATION,MD-88,2,142,0.0,Turbo-jet


# Insérez les données dans la table plane

In [10]:
data_to_insert = [
    (
        row['tailnum'], row['year'], row['type'], row['manufacturer'], row['model'], row['engines'], row['seats'], row['speed'], row['engine']
    )
    for index, row in df_planes.iterrows()
]

connection = get_db_connection()
cursor = connection.cursor()

cursor.executemany(
    "INSERT INTO plane (tailnum, year, type, manufacturer, model, engines, seats, speed, engine) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
    data_to_insert
)

connection.commit()
connection.close()

Connexion réussie à la base de données SQL Server.


# Importer les météo

In [11]:
reader = PdfReader("data/weather.pdf")
number_of_pages = len(reader.pages)
all_data = []
for page_num in range(number_of_pages):
    page = reader.pages[page_num]
    text = page.extract_text()
    lines = text.split('\n')
    if page_num == 0:
        lines = lines[1:]
    all_data.extend(lines)    
        
text_without_first_line = '\n'.join(all_data)

weather_data = [line.split(',') for line in text_without_first_line.split('\n') if line]
weather_data = pd.DataFrame(weather_data, columns=['origin', 'year', 'month', 'day', 'hour', 'temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'wind_gust', 'precip', 'pressure', 'visib', 'time_hour'])
# weather_data = weather_data.reset_index()
weather_data

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2021,1,1,0,37.04,21.92,53.97,230,10.357019999999999,11.918651475599997,0.0,1013.9,10.0,2021-01-01T00:00:00Z
1,EWR,2021,1,1,1,37.04,21.92,53.97,230,13.809359999999998,15.891535300799996,0.0,1013,10.0,2021-01-01T01:00:00Z
2,EWR,2021,1,1,2,37.94,21.92,52.09,230,12.658579999999999,14.567240692399997,0.0,1012.6,10.0,2021-01-01T02:00:00Z
3,EWR,2021,1,1,3,37.94,23,54.51,230,13.809359999999998,15.891535300799996,0.0,1012.7,10.0,2021-01-01T03:00:00Z
4,EWR,2021,1,1,4,37.94,24.08,57.04,240,14.960139999999999,17.215829909199996,0.0,1012.8,10.0,2021-01-01T04:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,EWR,2021,5,18,11,60.98,44.96,55.52,0,0,0,0.0,1025.5,10.0,2021-05-18T10:00:00Z
3295,EWR,2021,5,18,12,62.06,46.04,55.68,0,0,0,0.0,1025.8,10.0,2021-05-18T11:00:00Z
3296,EWR,2021,5,18,13,62.96,46.94,55.8,0,0,0,0.0,1026.2,10.0,2021-05-18T12:00:00Z
3297,EWR,2021,5,18,14,62.96,48.02,58.12,160,5.7539,6.621473041999999,0.0,1026.3,10.0,2021-05-18T13:00:00Z


# Insérez les données dans la table weather

In [12]:
data_to_insert = [
    (
        row['origin'], row['year'], row['month'], row['day'], row['hour'], row['temp'], row['dewp'], row['humid'], row['wind_dir'], row['wind_speed'], row['wind_gust'], row['precip'], row['pressure'], row['visib'], row['time_hour']
    )
    for index, row in weather_data.iterrows()
]

connection = get_db_connection()
cursor = connection.cursor()

cursor.executemany("INSERT INTO weather (origin, year, month, day, hour, temp, dewp, humid, wind_dir, wind_speed, wind_gust, precip, pressure, visib, time_hour) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", data_to_insert)

cursor.commit()
cursor.close()

Connexion réussie à la base de données SQL Server.
